In [ ]:
# Run this code first in order to initialize all the functions needed for the analysis

def Initialisation(root, name_outgdb):   
    # Allow to overwrite
    arcpy.env.overwriteOutput = True
    
    # Creation of the gdb file from ArcPy 
    outgdb = root + name_outgdb 
    
    if arcpy.Exists(root + outgdb):
        arcpy.Delete_management(outgdb)

    if not arcpy.Exists(root + outgdb):
        print("Creating", outgdb)
        arcpy.CreateFileGDB_management(root, name_outgdb)
        
def InsertData(data_workspace, root, outgdb):
    # Select the path where the data are stored
    arcpy.env.workspace = data_workspace
    
    # Insert in the output gdb file all the files needed for the analysis 
    arcpy.RasterToGeodatabase_conversion("Population_Density_2020.tif", root + outgdb)
    arcpy.FeatureClassToGeodatabase_conversion("SignificantVolcanicEruption.shp", root + outgdb)
    arcpy.FeatureClassToGeodatabase_conversion("TectonicPlate.shp", root + outgdb)
    

def SetParameters(workspace):
    arcpy.env.workspace = workspace
    
    # Addition of the column necessary for the analysis in the volcanoes table
    arcpy.AddField_management("SignificantVolcanicEruption","Next_Eruption","FLOAT")
    arcpy.AddField_management("SignificantVolcanicEruption","Distance_VEI","TEXT")
    arcpy.AddField_management("TectonicPlate","Presence","FLOAT")

    # Calculation of the distance depending on the VEI and the next possible eruption
    DistanceInsertion("SignificantVolcanicEruption", "Distance_VEI")
    NextEruption("SignificantVolcanicEruption","Next_Eruption")
    arcpy.CalculateField_management("TectonicPlate","Presence","100")
    
def DistanceInsertion(name_table, name_column):
    expression = "getDistanceVEI(!VEI!)"
    codeblock = """
def getDistanceVEI(VEI):
        if VEI == 1:
            return "1 Kilometers"
        elif VEI == 2:
            return "5 Kilometers"
        elif VEI == 3:
            return "15 Kilometers"
        elif VEI == 4:
            return "25 Kilometers"
        elif VEI == 5:
            return "35 Kilometers"
        elif VEI == 6:
            return "40 Kilometers"
        elif VEI == 7:
            return "50 Kilometers"
    """
    # Calculation of the column 
    arcpy.CalculateField_management(name_table,name_column,expression,"PYTHON3",codeblock)

def NextEruption(name_table,name_column):
    expression = "getNextEruption(!Year!,!VEI!)"
    codeblock = """
def getNextEruption(Year, VEI):
        if VEI == 1:
            return (2020 - Year - 1/365)
        if VEI == 2:
            return (2020 - Year - 1/52)
        if VEI == 3:
            return (2020 - Year - 1)
        if VEI == 4:
            return (2020 - Year - 10)
        if VEI == 5:
            return (2020 - Year - 100)
        if VEI == 6:
            return (2020 - Year - 100)
        if VEI == 7:
            return (2020 - Year - 1000)
    """
    # Calculation of the column
    arcpy.CalculateField_management(name_table,name_column,expression,"PYTHON3",codeblock)
    
def BufferAndRasterCreation(root, name_outgdb):
    arcpy.env.workspace = root + name_outgdb

    # Create a buffer around the volcano depending on the volcano VEI 
    arcpy.Buffer_analysis("SignificantVolcanicEruption", "Buffer_Table_Volcanoes", "Distance_VEI")

    # Create a Raster Layer depending on either the next eruption or the elevation of the volcano
    arcpy.PolygonToRaster_conversion("Buffer_Table_Volcanoes","Elevation","Raster_Volcano_Elevation", "CELL_CENTER", "", 250)
    arcpy.PolygonToRaster_conversion("Buffer_Table_Volcanoes","Next_Eruption","Raster_Volcano_Next_Eruption", "CELL_CENTER", "", 250)

    # Create a buffer around the tectonic plates depending on the user input
    Distance_user = input("DISTANCE FROM THE TECTONIC PLATES : ")
    arcpy.Buffer_analysis("TectonicPlate", "Buffer_File_Tectonic_Plates", Distance_user)

    # Create a raster layer in order to be analyze with the other layers 
    arcpy.PolygonToRaster_conversion("Buffer_File_Tectonic_Plates","Presence","Raster_Tectonic_Plates", "CELL_CENTER", "", 250)

In [ ]:
# LIBRARY
import arcpy

# EXTENSION
arcpy.CheckOutExtension("Spatial")

# WORSPACE ENVIRONMENT
root = "C:/Users/mletaill/Documents/ArcGIS/Projects/MyProjectV1/"
name_outgdb = "out_gdb.gdb"
arcpy.env.workspace = root + name_outgdb

# Creation of the output file for the analysis 
Initialisation(root, name_outgdb)
InsertData("C:/Users/mletaill/Documents/RUDN/DATA_PROJECT", root, name_outgdb)

# When running the code, an AttributeError can happen, such as : 'ToolValidator' object has 
# no attribute 'isLicensed'. It is a ArcGIS Pro problem, the program will continue to run anyway. 

# Create and fill the new columns needed for the volcanoes data
SetParameters(root + name_outgdb)

# Create the 2 buffers needed and the Rasters for the analysis
BufferAndRasterCreation(root, name_outgdb)

# Create a new raster that is the sum of the 2 rasters 
Raster_Volcanoes_Risks_Population = arcpy.sa.Plus("Raster_Volcano_Elevation", "Population_Density_2020")
Raster_Volcanoes_Risks_Population.save(root + name_outgdb + "/" + "Raster_Volcanoes_Risks_Population")